In [63]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [37]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\kunaual\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## Get article titles and teaser text 

In [3]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [30]:
html = browser.html  #get all the html
soup = BeautifulSoup(html, 'html.parser')

#testing output, see what to get
#print(soup.prettify())
 

titles=[]
teaser_texts=[]

divs = soup.find_all('div')
for div in divs:
    #print(div['class'][0])  #this will get you content_title and article_teaser_body
    if div['class'][0]=='content_title':
        #print(div.text)
        titles.append(div.text)
    if div['class'][0]=='article_teaser_body':
        #print(div.text)
        teaser_texts.append(div.text)



#titles = soup.find_all('h3')
#solution finds h3, as above 
#then does link=title.find('a')
#then href=link['href']
#then title=link['title']
# for title in titles:
#     print(f"Page:{x} ----------------")
#     print(title.a['title'])
#     print(title.a['href'])

## Get featured image url

In [38]:
iurl = 'https://spaceimages-mars.com/'
browser.visit(iurl)

In [61]:
ihtml = browser.html
isoup = BeautifulSoup(ihtml, 'html.parser')

#checkout html output, looking for featured image 
#print(isoup.prettify())

# looking for this src:
# <img class="headerimage fade-in" src="image/featured/mars1.jpg">

imgs = isoup.find_all('img')
for img in imgs:
    #print(img) #testing message
    try:
        if img['class'][0]=='headerimage':
            #print(img['src'])  #testing message
            featured_image_file = img['src']
            break   #header image will be towards the top, once found don't need to continue the loop
    except:  #if there's no class, continue to the next one
        continue

#concat the url to the image location
featured_image_url = iurl+featured_image_file

In [62]:
#check the url value
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

## Get Mars facts

In [65]:
turl = 'https://galaxyfacts-mars.com/'

In [67]:
tables = pd.read_html(turl)
len(tables)

2

In [ ]:
browser.quit()